In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from numpy import linalg as LA

In [2]:
#read data into dataframe
df = pd.read_csv('data/user_artists_small.txt', sep='\t')

In [3]:
df.head()

,userID,artistID,interactions
0,0,45,13883
1,0,46,11690
2,0,47,11351
3,0,48,10300
4,0,49,8983


In [4]:
#count unique users and artists
n_users = df['userID'].unique().shape[0]
n_items = df['artistID'].unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of artists = ' + str(n_items))

Number of users = 74 | Number of artists = 72


In [5]:
#pivot table into matrix fill NaN with 0
df_pivot = df.pivot(index='userID', columns='artistID', values='interactions').fillna(0)

In [6]:
#fill in the missing user and artist IDs between 1 and 100
df_pivot = df_pivot.reindex(index=range(0,100), columns=range(0,100), fill_value=0)

In [7]:
df_pivot

artistID,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1342.0,1337.0,1332.0,1330.0,1315.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13176.0,662.0,493.0,431.0,403.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1513.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#count the nonzero elements in the matrix
nonzero = np.count_nonzero(df_pivot.values)
sparsity = nonzero/(100*100-nonzero)
print("sparsity is :" + str(sparsity))

sparsity is :0.027221366204417053


In [27]:
from scipy.sparse.linalg import spsolve
from scipy import sparse

In [10]:
m = 100
n = 100
f = 3
lamda = 0.01

X = sparse.csr_matrix(np.zeros((m, f)) + 0.5)  
Y = sparse.csr_matrix(np.zeros((n, f))) 

In [11]:
lamdaI = sparse.csr_matrix(np.eye(f)*lamda)

In [12]:
r_ui = df_pivot.values
c_ui = np.ones((m,n))
c_ui = np.where(r_ui > 0, 1 + sparsity*r_ui, 1)
p_ui = np.where(r_ui > 0, 1, 0)

In [37]:
print(c_ui.shape)

(100, 100)


In [14]:
P = p_ui
C = c_ui

X = np.zeros((m, f)) + 0.5
Y = np.zeros((n, f))

In [32]:
m = 100
n = 100
f = 3
lamda = 0.01

# TODO UNCOMMENT WHEN READY TO TEST!
X = sparse.csr_matrix(np.zeros((m, f)) + 0.5)  
Y = sparse.csr_matrix(np.zeros((n, f))) 
# X = np.zeros((m, f)) + 0.5
# Y = np.zeros((n, f))

# P = p_ui
# C = c_ui
P = sparse.csr_matrix(p_ui)
C = sparse.csr_matrix(c_ui)

for i in range(1):
    
    xTx = X.T.dot(X)
    for item in range(n):
        p_i = P[:, item]
        # the sparse matrix: C_i - I
        C_i_I = sparse.linalg.eigs(C[:, item], k =99, which='LM', return_eigenvectors=False)
        #C_i_I = np.diag(C[:,item])
        # C_i_I = np.diag(C[:,item])
        left = xTx + lamdaI
        right = X.T.dot(C_i_I).dot(p_i)
        print(right.shape)
        pass
        # y_i = np.linalg.solve(left,right)
        # print(y_i)
        y_i = spsolve(left, right)
        # #print(y_i)
        Y[item] = y_i
        #print(Y[item])

    #update user matrix
    yTy = Y.T.dot(Y)
    for user in range(m):
        p_u = P[user, :]
        C_u_I = np.diag(C[user,:])
        left = yTy + lamdaI
        right = Y.T.dot(C_u_I).dot(p_u.T)
        x_u = np.linalg.solve(left, right)
        X[user] = x_u


ValueError: expected square matrix (shape=(100, 1))

In [367]:
lamdaI = np.eye(f)*lamda

In [368]:
cost = (np.power(P - X.dot(Y.T), 2) * C).sum() + lamda * (np.power(X, 2).sum() + np.power(Y, 2).sum())

In [369]:
cost

49645283.82948818

In [370]:
cost = np.sum(c_ui*((p_ui - X.dot(Y.transpose()))**2)) + lamda*(np.sum(LA.norm(X, axis = 1)**2)+np.sum(LA.norm(Y, axis = 1)**2))

In [371]:
cost

49645283.82948818